## Confifure Azure computing resources

In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="hyperdrive-exp")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130418
Azure region: southcentralus
Subscription id: b19c9c81-5f59-4537-b3c0-c1beb163ec22
Resource group: aml-quickstarts-130418


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Create compute cluster
from azureml.core.compute_target import ComputeTargetException

vm_size = "Standard_D12_V2"


# The name for the CPU cluster
cpu_cluster = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Configure hyperdrive tuning parameters 

In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
        "--C": uniform(0.1,1),
        "--max_iter": choice(50, 100, 150, 200)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)### YOUR CODE HERE ###

# Create a SKLearn estimator for use with train.py
#shutil.copy('train-Copy1.py', "./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.', compute_target=cpu_cluster, 
              entry_script='train.py')### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy= policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=100,
                                max_concurrent_runs=5)
                                ### YOUR CODE HERE###

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

###HyperDriveRun(experiment, run_id, hyperdrive_config=hyperdrive_config)
hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_5a5b986d-4d8f-4dc7-b696-34ee414a79a7
Web View: https://ml.azure.com/experiments/hyperdrive-exp/runs/HD_5a5b986d-4d8f-4dc7-b696-34ee414a79a7?wsid=/subscriptions/b19c9c81-5f59-4537-b3c0-c1beb163ec22/resourcegroups/aml-quickstarts-130418/workspaces/quick-starts-ws-130418

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-14T02:03:33.336352][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2020-12-14T02:03:33.640010][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-14T02:03:34.7013038Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-14T02:03:32.770641][API][INFO]Experiment created<END>\n"


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [9]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best run ID: ', best_run.id)
print('\n Metrics: ', best_run_metrics)
print('\n Parameters: ', parameter_values)

Best run ID:  HD_5a5b986d-4d8f-4dc7-b696-34ee414a79a7_1

 Metrics:  {'Regularization Strength:': 0.39976389985209604, 'Max iterations:': 150, 'Accuracy': 0.7575757575757576}

 Parameters:  ['--C', '0.39976389985209604', '--max_iter', '150']


In [10]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
hyperdrive-exp,HD_5a5b986d-4d8f-4dc7-b696-34ee414a79a7_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
best_run.id

'HD_5a5b986d-4d8f-4dc7-b696-34ee414a79a7_1'

In [12]:
os.makedirs('outputs', exist_ok = True)
joblib.dump(value = best_run.id, filename = 'outputs/hyperdrive_model.joblib')

['outputs/hyperdrive_model.joblib']

In [13]:
# check 
best_run.get_file_names()

['azureml-logs/55_azureml-execution-tvmps_ec5b4de7cb03be18a840725c33f074d28df74244124a36ad2b71357b25f304c2_d.txt',
 'azureml-logs/65_job_prep-tvmps_ec5b4de7cb03be18a840725c33f074d28df74244124a36ad2b71357b25f304c2_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_ec5b4de7cb03be18a840725c33f074d28df74244124a36ad2b71357b25f304c2_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'logs/azureml/102_azureml.log',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/hyperdrive_model.joblib',
 'outputs/model.joblib']

In [17]:
# register the best model 
#best_run.register_model(model_name = 'best_regression_model', model_path = 'outputs/model.joblib')